# RESTAURANT - Feature Engineering and Feature Selection

In [1]:
from scipy import stats
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import os
import statsmodels.api as sm
import matplotlib.transforms as mtransforms
from scipy import stats
import warnings
from statsmodels.tsa.stattools import adfuller
from IPython.display import Image
import logging
from fbprophet import Prophet
import warnings
warnings.simplefilter('ignore')
import multiprocessing
from multiprocessing import Pool

warnings.simplefilter('ignore')

In [2]:
from sklearn import preprocessing

In [3]:
# global vars
# plt.rcParams.keys()
plt.rcParams['grid.linestyle'] ='--'
plt.rcParams['grid.alpha'] = 0.4
plt.rcParams['figure.figsize'] = [7,3]
plt.rcParams["font.weight"] = "light"
plt.rcParams["axes.titleweight"] = "bold"

sns.set_palette("PRGn", 10)

In [4]:
%matplotlib inline

In [5]:
from IPython.display import display_html

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

# IMPORT DATA

In [6]:
path_input = ('../input/')

In [7]:
!ls -1 {path_input}

air_reserve.csv.gz
air_store_info.csv.gz
air_visit_data.csv.gz
date_info.csv.gz
hpg_reserve.csv.gz
hpg_store_info.csv.gz
sample_submission.csv.gz
store_id_relation.csv.gz


In [8]:
air_reserve = pd.read_csv(os.path.join(path_input,'air_reserve.csv.gz'))
air_store_info = pd.read_csv(os.path.join(path_input,'air_store_info.csv.gz'))
air_visit_data = pd.read_csv(os.path.join(path_input,'air_visit_data.csv.gz'))
date_info = pd.read_csv(os.path.join(path_input,'date_info.csv.gz'))
hpg_reserve = pd.read_csv(os.path.join(path_input,'hpg_reserve.csv.gz'))
hpg_store_info = pd.read_csv(os.path.join(path_input,'hpg_store_info.csv.gz'))
sample_submission = pd.read_csv(os.path.join(path_input,'sample_submission.csv.gz'))
store_id_relation = pd.read_csv(os.path.join(path_input,'store_id_relation.csv.gz'))

# Feature engineering

#### Feature engineering steps:

create a seperate dataframe for feature enginnering (say train and test)

for each feature:
- create feature
- rename columns
- merge on train test dfs

# VISIT TABLE

In [9]:
train = air_visit_data.copy()
test = sample_submission.copy()

In [10]:
test['air_store_id'] = test['id'].apply(lambda x: '_'.join(x.split('_')[:2]))

test['visit_date'] = test['id'].apply(lambda x: x.split('_')[2])

In [11]:
train.air_store_id.nunique(),test.air_store_id.nunique()

(829, 821)

In [12]:
#drop id's not in test
train = train[train.air_store_id.isin(test.air_store_id.unique())]

## convert date format and create some date features

In [13]:
train['visit_date'] = pd.to_datetime(train['visit_date'])
train['year'] = train['visit_date'].dt.year
train['dow'] =  train['visit_date'].dt.dayofweek
train['dom'] = train['visit_date'].dt.day
train['month'] = train['visit_date'].dt.month
train['visit_date'] = train['visit_date'].dt.date

In [14]:
test['visit_date'] = pd.to_datetime(test['visit_date'])
test['year'] = test['visit_date'].dt.year
test['dow'] = test['visit_date'].dt.dayofweek
test['dom'] = test['visit_date'].dt.day
test['month'] = test['visit_date'].dt.month
test['visit_date'] = test['visit_date'].dt.date

    Some restaurants at train dataset does not exit on test

In [16]:
len(set(train['air_store_id'].unique()) - set(test['air_store_id'].unique()))

0

## VISIT / DATE FEATURES

#### Day of week

In [18]:
aggregations = ['mean','median','min','max','std']

gr = train.groupby('dow')['visitors'].agg(aggregations)

gr.columns = ['dow_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

In [19]:
train = train.merge(gr,on='dow',how='left')

test = test.merge(gr,on='dow',how='left')

#### Day of month

In [20]:
aggregations = ['mean', 'median', 'min', 'max', 'std']

gr = train.groupby('dom')['visitors'].agg(aggregations)

gr.columns = ['dom_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

train = train.merge(gr, on='dom', how='left')

test = test.merge(gr, on='dom', how='left')


#### Month of year

In [21]:
aggregations = ['mean', 'median', 'min', 'max', 'std']

gr = train.groupby('month')['visitors'].agg(aggregations)

gr.columns = ['month_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

train = train.merge(gr, on='month', how='left')

test = test.merge(gr, on='month', how='left')

In [22]:
train.head()

,air_store_id,visit_date,visitors,year,dow,dom,month,dow_mean,dow_median,dow_min,...,dom_mean,dom_median,dom_min,dom_max,dom_std,month_mean,month_median,month_min,month_max,month_std
0,air_ba937bf13d40fb24,2016-01-13,25,2016,2,13,1,19.217106,16,1,...,19.254301,15,1,269,15.294143,19.968233,16,1,514,16.041859
1,air_ba937bf13d40fb24,2016-01-14,32,2016,3,14,1,18.898359,15,1,...,20.194978,16,1,372,15.855515,19.968233,16,1,514,16.041859
2,air_ba937bf13d40fb24,2016-01-15,29,2016,4,15,1,23.087137,19,1,...,21.241133,18,1,150,16.096449,19.968233,16,1,514,16.041859
3,air_ba937bf13d40fb24,2016-01-16,22,2016,5,16,1,26.310500,22,1,...,21.392962,17,1,207,16.755058,19.968233,16,1,514,16.041859
4,air_ba937bf13d40fb24,2016-01-18,6,2016,0,18,1,17.170329,14,1,...,21.869031,18,1,777,18.646356,19.968233,16,1,514,16.041859


## STORE FEATURES

#### Visitors by stores and days of weeks

In [23]:
stores_features = []

gr = train.groupby(['air_store_id','dow'])['visitors'].agg(['min','mean','median','max','count','std'])
   
gr = gr.unstack(level=-1).fillna(0)

gr.columns = ['STORE_VISITORS_BY_DATE_{}_{}'.format(e[0],e[1]) for e in gr.columns]

stores_features.extend(gr.columns)

gr = gr.reset_index()

gr.head()

,air_store_id,STORE_VISITORS_BY_DATE_min_0,STORE_VISITORS_BY_DATE_min_1,STORE_VISITORS_BY_DATE_min_2,STORE_VISITORS_BY_DATE_min_3,STORE_VISITORS_BY_DATE_min_4,STORE_VISITORS_BY_DATE_min_5,STORE_VISITORS_BY_DATE_min_6,STORE_VISITORS_BY_DATE_mean_0,STORE_VISITORS_BY_DATE_mean_1,...,STORE_VISITORS_BY_DATE_count_4,STORE_VISITORS_BY_DATE_count_5,STORE_VISITORS_BY_DATE_count_6,STORE_VISITORS_BY_DATE_std_0,STORE_VISITORS_BY_DATE_std_1,STORE_VISITORS_BY_DATE_std_2,STORE_VISITORS_BY_DATE_std_3,STORE_VISITORS_BY_DATE_std_4,STORE_VISITORS_BY_DATE_std_5,STORE_VISITORS_BY_DATE_std_6
0,air_00a91d42b08b08d9,1.0,1.0,15.0,15.0,17.0,3.0,2.0,22.457143,24.350000,...,40.0,38.0,1.0,9.425693,9.093672,8.881521,7.143968,9.021342,16.793639,0.000000
1,air_0164b9927d20bcc3,2.0,1.0,2.0,1.0,3.0,1.0,0.0,7.500000,9.560000,...,28.0,22.0,0.0,5.226451,6.076457,6.271946,5.682903,6.914730,6.925860,0.000000
2,air_0241aa3964b7f861,2.0,1.0,2.0,1.0,1.0,2.0,1.0,8.920635,8.621212,...,61.0,65.0,64.0,4.790116,5.524115,4.902831,6.227181,6.762925,7.235164,5.826782
3,air_0328696196e46f18,2.0,2.0,1.0,1.0,1.0,2.0,1.0,6.416667,7.285714,...,22.0,17.0,17.0,7.115391,4.339950,5.973594,7.139369,4.244681,6.239697,10.607295
4,air_034a3d5b40d5b1b1,1.0,1.0,1.0,1.0,2.0,1.0,1.0,11.864865,10.888889,...,41.0,40.0,38.0,11.576226,8.594724,19.667217,8.480588,8.225629,12.134899,15.973841


In [24]:
train = pd.merge(train, gr, how='left', on=['air_store_id']) 
test = pd.merge(test, gr, how='left', on=['air_store_id']) 

# STORE TABLE

## Label encode genres and areas

In [25]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


## Genre

In [26]:
air_store_info.air_genre_name.unique()

array(['Italian/French', 'Dining bar', 'Yakiniku/Korean food',
       'Cafe/Sweets', 'Izakaya', 'Okonomiyaki/Monja/Teppanyaki',
       'Bar/Cocktail', 'Japanese food', 'Creative cuisine', 'Other',
       'Western food', 'International cuisine', 'Asian', 'Karaoke/Party'],
      dtype=object)

In [27]:
#init label encoder
lbl = preprocessing.LabelEncoder()
# label encode genre
air_store_info['air_genre_name'] = lbl.fit_transform(air_store_info['air_genre_name'])

In [28]:
train = pd.merge(train, air_store_info[['air_store_id', 'air_genre_name']], how='left', on=['air_store_id']) 
test = pd.merge(test, air_store_info[['air_store_id', 'air_genre_name']], how='left', on=['air_store_id'])

#### Aggregations with genre

In [29]:
aggregations = ['mean', 'median', 'min', 'max', 'std']

gr = train.groupby('air_genre_name')['visitors'].agg(aggregations)

gr.columns = ['genre_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

In [30]:
train = train.merge(gr, on='air_genre_name',how='left')
test = test.merge(gr, on='air_genre_name',how='left')

## AREA

In [31]:
for e in np.sort(air_store_info['air_area_name'].unique()):
    print(e)

Fukuoka-ken Fukuoka-shi Daimyō
Fukuoka-ken Fukuoka-shi Hakata Ekimae
Fukuoka-ken Fukuoka-shi Imaizumi
Fukuoka-ken Fukuoka-shi Momochi
Fukuoka-ken Fukuoka-shi Shiobaru
Fukuoka-ken Fukuoka-shi Takatori
Fukuoka-ken Fukuoka-shi Tenjin
Fukuoka-ken Fukuoka-shi Torikai
Fukuoka-ken Itoshima-shi Maebarunishi
Fukuoka-ken Kitakyūshū-shi Konyamachi
Fukuoka-ken Kitakyūshū-shi None
Fukuoka-ken Kitakyūshū-shi Ōtemachi
Fukuoka-ken Kurume-shi Jōnanmachi
Fukuoka-ken Yame-shi Motomachi
Hiroshima-ken Fukuyama-shi Higashisakuramachi
Hiroshima-ken Hiroshima-shi Kokutaijimachi
Hiroshima-ken Hiroshima-shi Nagarekawachō
Hokkaidō Abashiri-shi Minami 6 Jōhigashi
Hokkaidō Asahikawa-shi 6 Jōdōri
Hokkaidō Katō-gun Motomachi
Hokkaidō Sapporo-shi Atsubetsuchūō 1 Jō
Hokkaidō Sapporo-shi Kita 24 Jōnishi
Hokkaidō Sapporo-shi Kotoni 2 Jō
Hokkaidō Sapporo-shi Minami 3 Jōnishi
Hyōgo-ken Amagasaki-shi Higashinanamatsuchō
Hyōgo-ken Himeji-shi Yasuda
Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike
Hyōgo-ken Kōbe-shi Kumoidōri
Hy

In [32]:
# http://www.wikizeroo.net/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvQWRtaW5pc3RyYXRpdmVfZGl2aXNpb25zX29mX0phcGFu

In [33]:
air_store_info['air_area_name'] = air_store_info['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))

In [34]:
air_store_info['air_area_name_1'] = air_store_info['air_area_name'].apply(lambda x: x.split()[0])

air_store_info['air_area_name_2'] = air_store_info['air_area_name'].apply(lambda x: x.split()[1])

air_store_info['air_area_name_3'] = air_store_info['air_area_name'].apply(lambda x: x.split()[2])

air_store_info['air_area_name_4'] = air_store_info['air_area_name'].apply(lambda x: x.split()[3])

air_store_info['air_area_name_5'] = air_store_info['air_area_name'].apply(lambda x: '_'.join(x.split()[3:]))

In [35]:
for e in [cols for cols in air_store_info.columns if 'air_area_name' in cols]:
        lbl = preprocessing.LabelEncoder()
        air_store_info[e] = lbl.fit_transform(air_store_info[e])

In [37]:
cols_area = [cols for cols in air_store_info.columns if 'air_area_name' in cols]

In [38]:
train = pd.merge(train, air_store_info[['air_store_id']+cols_area], how='left', on=['air_store_id']) 
test = pd.merge(test, air_store_info[['air_store_id']+cols_area], how='left', on=['air_store_id'])

# COORDINATE FEATURES

In [39]:
air_store_info = pd.read_csv(os.path.join(path_input,'air_store_info.csv.gz'))

In [40]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(air_store_info[['longitude','latitude']])
air_store_info['loc_cluster'] = kmeans.predict(air_store_info[['longitude','latitude']])

In [41]:
# for _,e in air_store_info.groupby('loc_cluster'):
#     display(e)

In [42]:
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
cluster_centers = cluster_centers.reset_index()
cluster_centers.columns = ['loc_cluster','center_x','center_y'] 

In [43]:
air_store_info = air_store_info.merge(cluster_centers,on='loc_cluster',how='left')

In [44]:
air_store_info['loc_distance_to_centroid'] = np.linalg.norm(air_store_info[['longitude','latitude']].values-
               air_store_info[['center_x','center_y']].values,axis=1)


In [45]:
loc_cols = [cols for cols in air_store_info if 'loc' in cols]

In [46]:
train = pd.merge(train, air_store_info[['air_store_id']+loc_cols], how='left', on=['air_store_id']) 
test = pd.merge(test, air_store_info[['air_store_id']+loc_cols], how='left', on=['air_store_id'])

# RESERVATION FEAUTURES

In [47]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [48]:
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['visit_datetime'] = air_reserve['visit_datetime'].dt.date
air_reserve['reserve_datetime'] = air_reserve['reserve_datetime'].dt.date
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda x: (x['visit_datetime'] - x['reserve_datetime']).days, axis=1)

In [49]:
# initialize a list to store column names 
air_reserve_features = []
# aggregation list to aggregate
aggregations =  ['mean', 'min', 'max', 'sum', 'std', 'median']
# groupby and aggregataions
gr = air_reserve.groupby(['air_store_id','visit_datetime'])\
[['reserve_datetime_diff', 'reserve_visitors']].agg(aggregations)
# change column names
gr.columns = ['air_reserve_{}_{}'.format(e[0],e[1]) for e in gr.columns]
# add column nmes 
air_reserve_features.extend(gr.columns)
# prepare for merge
gr = gr.reset_index()
gr = gr.rename(columns={'visit_datetime':'visit_date'})

gr.head(2)

,air_store_id,visit_date,air_reserve_reserve_datetime_diff_mean,air_reserve_reserve_datetime_diff_min,air_reserve_reserve_datetime_diff_max,air_reserve_reserve_datetime_diff_sum,air_reserve_reserve_datetime_diff_std,air_reserve_reserve_datetime_diff_median,air_reserve_reserve_visitors_mean,air_reserve_reserve_visitors_min,air_reserve_reserve_visitors_max,air_reserve_reserve_visitors_sum,air_reserve_reserve_visitors_std,air_reserve_reserve_visitors_median
0,air_00a91d42b08b08d9,2016-10-31,0.0,0,0,0,NaN,0.0,2.0,2,2,2,NaN,2.0
1,air_00a91d42b08b08d9,2016-12-05,4.0,4,4,4,NaN,4.0,9.0,9,9,9,NaN,9.0


In [50]:
train = train.merge(gr, how='left', on=['air_store_id','visit_date'])
test = test.merge(gr, how='left', on=['air_store_id','visit_date'])

In [51]:
# sanity check
train[(train.air_store_id=='air_fee8dcf4d619598e')&(train.visit_date == pd.to_datetime('2017-01-09').date())]

,air_store_id,visit_date,visitors,year,dow,dom,month,dow_mean,dow_median,dow_min,...,air_reserve_reserve_datetime_diff_max,air_reserve_reserve_datetime_diff_sum,air_reserve_reserve_datetime_diff_std,air_reserve_reserve_datetime_diff_median,air_reserve_reserve_visitors_mean,air_reserve_reserve_visitors_min,air_reserve_reserve_visitors_max,air_reserve_reserve_visitors_sum,air_reserve_reserve_visitors_std,air_reserve_reserve_visitors_median
35703,air_fee8dcf4d619598e,2017-01-09,17,2017,0,9,1,17.170329,14,1,...,13.0,16.0,7.071068,8.0,2.5,2.0,3.0,5.0,0.707107,2.5


###    do the same for hpg tables

In [52]:
hpg_reserve.shape

(2000320, 4)

In [53]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [54]:
# first, merge hpg_reserve and store_id table to put air_reserve_id info
store_id_relation.head(2)

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809


In [55]:
# we are losing lots of information since we keep less than 0.02 rows of the hpg_reserves 
# table!

# how='inner' keep restaurants that only hve both air and hpg ids
hpg_air_reserve = pd.merge(hpg_reserve,store_id_relation,how='inner', on=['hpg_store_id'])

In [56]:
hpg_air_reserve.shape

(28183, 5)

In [57]:
hpg_air_reserve.head(2)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926


    but we are losing lots of information since we keep less than 0.02 rows of the hpg_reserves table!

In [58]:
print(hpg_reserve.shape, hpg_air_reserve.shape, hpg_air_reserve.shape[0]/hpg_reserve.shape[0])

(2000320, 4) (28183, 5) 0.014089245720684691


In [59]:
hpg_air_reserve['visit_datetime'] = pd.to_datetime(hpg_air_reserve['visit_datetime'])
hpg_air_reserve['reserve_datetime'] = pd.to_datetime(hpg_air_reserve['reserve_datetime'])
hpg_air_reserve['reserve_datetime'] = hpg_air_reserve['reserve_datetime'].dt.date
hpg_air_reserve['visit_datetime'] = hpg_air_reserve['visit_datetime'].dt.date
hpg_air_reserve['reserve_datetime_diff'] = hpg_air_reserve.apply(lambda x: (
    x['visit_datetime'] - x['reserve_datetime']).days, axis=1)

In [60]:
# initialize a list to store column names
hpg_air_reserve_features = []
# aggregation list to aggregate
aggregations = ['mean', 'min', 'max', 'sum', 'std', 'median']
# groupby and aggregataions
gr = hpg_air_reserve.groupby(['air_store_id', 'visit_datetime'])[
    ['reserve_datetime_diff', 'reserve_visitors']].agg(aggregations)
# change column names
gr.columns = ['hpg_air_reserve_{}_{}'.format(e[0], e[1]) for e in gr.columns]
# add column nmes
hpg_air_reserve_features.extend(gr.columns)
# prepare for merge
gr = gr.reset_index()
gr = gr.rename(columns={'visit_datetime': 'visit_date'})

gr.head(2)

,air_store_id,visit_date,hpg_air_reserve_reserve_datetime_diff_mean,hpg_air_reserve_reserve_datetime_diff_min,hpg_air_reserve_reserve_datetime_diff_max,hpg_air_reserve_reserve_datetime_diff_sum,hpg_air_reserve_reserve_datetime_diff_std,hpg_air_reserve_reserve_datetime_diff_median,hpg_air_reserve_reserve_visitors_mean,hpg_air_reserve_reserve_visitors_min,hpg_air_reserve_reserve_visitors_max,hpg_air_reserve_reserve_visitors_sum,hpg_air_reserve_reserve_visitors_std,hpg_air_reserve_reserve_visitors_median
0,air_00a91d42b08b08d9,2016-01-14,3.0,3,3,3,NaN,3.0,2.0,2,2,2,NaN,2.0
1,air_00a91d42b08b08d9,2016-01-15,6.0,6,6,6,NaN,6.0,4.0,4,4,4,NaN,4.0


In [61]:
train = train.merge(gr, how='left', on=['air_store_id','visit_date'])
test = test.merge(gr, how='left', on=['air_store_id','visit_date'])

In [62]:
# train.head()

#### Merge reserve and visit tables

In [64]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1,0
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3,0
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6,0
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2,0
4,air_db80363d35f10926,2016-01-01,2016-01-01,5,0


In [65]:
air_reserve = air_reserve.rename(columns={'visit_datetime':'visit_date'})

In [66]:
train = pd.merge(train, air_reserve, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, air_reserve, how='left', on=['air_store_id','visit_date'])

In [67]:
# train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

#### Holiday

In [68]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [69]:
date_info = date_info.rename(columns={'calendar_date': 'visit_date'})
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
# date_info['dow'] = lbl.fit_transform(date_info['day_of_week'])
date_info['visit_date'] = date_info['visit_date'].dt.date

In [70]:
train = pd.merge(train, date_info[['visit_date', 'holiday_flg']], how='left', on=['visit_date']) 
test = pd.merge(test, date_info[['visit_date', 'holiday_flg']], how='left', on=['visit_date']) 

# Use id as a feature

In [71]:
lbl = preprocessing.LabelEncoder()
train['le_air_store_id'] = lbl.fit_transform(train['air_store_id'])
test['le_air_store_id'] = lbl.transform(test['air_store_id'])

# col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
# train = train.fillna(-1)
# test = test.fillna(-1)

In [72]:
col = [c for c in train if c not in ['id','visitors']]

In [73]:
target = train['visitors']

In [74]:
train[col].to_pickle('./stored_files/24_12_train.pkl')

In [75]:
# train[col].isna().sum().sort_values(ascending=False)

In [76]:
test[col].to_pickle('./stored_files/24_12_test.pkl')

In [77]:
target.to_pickle('./stored_files/24_12_target.pkl')

### ref

https://machinelearningmastery.com/time-series-data-stationary-python/